# Addis Ababa: Meteorology × Source Apportionment Analysis

**Purpose**: Investigate how meteorological conditions interact with aerosol source types,
BC/EC measurement agreement, and concentration levels.

## Analyses

| Section | Analysis | Key Question |
|---------|----------|--------------|
| **1** | Precipitation Washout by Source | Does rain suppress biomass BC more than fossil fuel? |
| **2** | Wind Direction × Source Type | Do polluted marine/fossil days come from specific directions? |
| **3** | Temperature Range vs Concentration | Does atmospheric stability trap pollutants? |
| **4** | Humidity Effect on Measurement Agreement | Does RH explain HIPS vs FTIR EC scatter? |

### Data Sources
- **Hourly met**: Meteostat station 63450 (temp, dew point, RH, wind dir/speed, precip)
- **Daily met**: Aggregated daily averages (tavg/tmin/tmax, precip, wind speed, pressure)
- **Source apportionment**: PMF K_F concentrations matched to filter samples

---

## Section 0: Setup & Data Loading

In [ ]:
import sys
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from scipy import stats
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.projections.polar import PolarAxes
import matplotlib.cm as cm
import warnings
warnings.filterwarnings('ignore')

# Resolve repo paths for portable notebook execution
cwd = Path.cwd().resolve()
repo_root = next((p for p in [cwd, *cwd.parents] if (p / 'pyproject.toml').exists()), cwd)
notebook_dir = str((repo_root / 'notebooks').resolve())
data_root = Path(
    os.environ.get('AETHMODULAR_DATA_ROOT', str(repo_root / 'research' / 'ftir_hips_chem'))
).expanduser().resolve()

# Add research scripts folder to path
scripts_path = str((repo_root / 'research' / 'ftir_hips_chem' / 'scripts').resolve())
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
if scripts_path not in sys.path:
    sys.path.insert(0, scripts_path)

from config import SITES, MAC_VALUE
from data_matching import (
    load_aethalometer_data,
    load_filter_data,
    add_base_filter_id,
    match_all_parameters,
    load_etad_factors_with_filter_ids,
)

print(f'Repo root: {repo_root}')
print(f'Data root: {data_root}')
print(f'MAC value: {MAC_VALUE} m²/g')


In [ ]:
# =============================================================================
# Configuration (consistent with friday_summary_consolidated.ipynb)
# =============================================================================

ADDIS_CONFIG = {
    'name': 'Addis_Ababa',
    'code': 'ETAD',
    'timezone': 'Africa/Addis_Ababa',
}

# Ethiopian seasons
SEASONS = {
    'Dry Season': [10, 11, 12, 1, 2],
    'Belg Rainy Season': [3, 4, 5],
    'Kiremt Rainy Season': [6, 7, 8, 9],
}
SEASONS_ORDER = ['Dry Season', 'Belg Rainy Season', 'Kiremt Rainy Season']
SEASON_COLORS = {
    'Dry Season': '#E67E22',
    'Belg Rainy Season': '#27AE60',
    'Kiremt Rainy Season': '#3498DB',
}

def get_season_3(month):
    for season, months in SEASONS.items():
        if month in months:
            return season
    return None

# Source categories — standardized colors
SOURCE_CATEGORIES = {
    'charcoal':        {'label': 'Charcoal Burning',       'color': '#2C3E50', 'marker': 'o'},
    'wood':            {'label': 'Wood Burning',           'color': '#8B4513', 'marker': 's'},
    'fossil_fuel':     {'label': 'Fossil Fuel Combustion', 'color': '#7D3C98', 'marker': '^'},
    'polluted_marine': {'label': 'Polluted Marine',        'color': '#2980B9', 'marker': 'D'},
    'sea_salt':        {'label': 'Sea Salt Mixed',         'color': '#1ABC9C', 'marker': 'v'},
}
SOURCE_ORDER = ['charcoal', 'wood', 'fossil_fuel', 'polluted_marine', 'sea_salt']

GROUPED_SOURCE_CATEGORIES = {
    'biomass_burning':        {'label': 'Biomass Burning',        'color': '#556B2F', 'marker': 'o'},
    'fossil_fuel_combustion': {'label': 'Fossil Fuel Combustion', 'color': '#8B0000', 'marker': '^'},
    'sea_salt':               {'label': 'Sea Salt Mixed',         'color': '#1ABC9C', 'marker': 'v'},
}
GROUPED_SOURCE_ORDER = ['biomass_burning', 'fossil_fuel_combustion', 'sea_salt']

# Column mappings
FACTOR_TO_FRAC = {
    'GF3 (Charcoal)':              'charcoal_frac',
    'GF2 (Wood Burning)':          'wood_frac',
    'GF5 (Fossil Fuel Combustion)':'fossil_fuel_frac',
    'GF4 (Polluted Marine)':       'polluted_marine_frac',
    'GF1 (Sea Salt Mixed)':        'sea_salt_frac',
}
FACTOR_TO_CONC = {
    'K_F3 Charcoal (ug/m3)':              'charcoal_conc',
    'K_F2 Wood Burning (ug/m3)':           'wood_conc',
    'K_F5 Fossil Fuel Combustion (ug/m3)': 'fossil_fuel_conc',
    'K_F4 Polluted Marine (ug/m3)':        'polluted_marine_conc',
    'K_F1 Sea Salt Mixed (ug/m3)':         'sea_salt_conc',
}

CONC_TO_SOURCE = {
    'charcoal_conc':        'charcoal',
    'wood_conc':            'wood',
    'fossil_fuel_conc':     'fossil_fuel',
    'polluted_marine_conc': 'polluted_marine',
    'sea_salt_conc':        'sea_salt',
}

frac_cols = list(FACTOR_TO_FRAC.values())
conc_cols = list(FACTOR_TO_CONC.values())

# Font configuration (increased)
FONT = {'title': 16, 'axis': 14, 'tick': 12, 'legend': 11, 'annot': 11}
plt.rcParams.update({
    'font.size': FONT['tick'],
    'axes.titlesize': FONT['title'],
    'axes.labelsize': FONT['axis'],
    'legend.fontsize': FONT['legend'],
})

# Output directories
output_root = repo_root / 'artifacts' / 'notebook_outputs' / 'meteorology_source_interaction'
dirs = {}
for d in ['plots', 'data', 'friday_slides']:
    path = output_root / d
    path.mkdir(parents=True, exist_ok=True)
    dirs[d] = str(path)

print(f'Configuration ready. Outputs: {output_root}')

In [ ]:
# =============================================================================
# Load Meteorological Data
# =============================================================================

def resolve_met_path(*filenames):
    search_dirs = [
        data_root / 'Weather Data' / 'Meteostat',
        Path(notebook_dir),
    ]
    for base in search_dirs:
        for name in filenames:
            candidate = base / name
            if candidate.exists():
                return candidate
    search_text = ', '.join(str(d) for d in search_dirs)
    files_text = ', '.join(filenames)
    raise FileNotFoundError(f'Could not find [{files_text}] under: {search_text}')


# --- Daily met data ---
MET_DAILY_PATH = resolve_met_path(
    'Addis_Ababa_daily_Average_met_Data.csv',
    'Addis Ababa daily Average met Data.csv',
)
met_daily = pd.read_csv(MET_DAILY_PATH, encoding='utf-8-sig')
met_daily['date'] = pd.to_datetime(met_daily['date'])
met_daily['date_only'] = met_daily['date'].dt.normalize()

# Derived columns
met_daily['temp_range'] = met_daily['tmax'] - met_daily['tmin']  # diurnal temperature range
met_daily['Month'] = met_daily['date'].dt.month
met_daily['season'] = met_daily['Month'].apply(get_season_3)

print(f'Daily met file: {MET_DAILY_PATH}')
print(f"Daily met: {len(met_daily)} days, {met_daily['date'].min().date()} to {met_daily['date'].max().date()}")
print(f"  Columns: {list(met_daily.columns)}")
print(f"  Precip coverage: {met_daily['prcp'].notna().sum()} days")
print(f"  Pressure coverage: {met_daily['pres'].notna().sum()} days")

# --- Hourly met data ---
MET_HOURLY_PATH = resolve_met_path('master_meteostat_AddisAbaba_63450_2022-12-01_2024-10-01.csv')
met_hourly = pd.read_csv(MET_HOURLY_PATH)
met_hourly = met_hourly.replace('NA', np.nan)
met_hourly['time'] = pd.to_datetime(met_hourly['time'])

# Convert numeric columns
for col in ['temp', 'dwpt', 'rhum', 'prcp', 'wdir', 'wspd', 'pres']:
    met_hourly[col] = pd.to_numeric(met_hourly[col], errors='coerce')

met_hourly['date_only'] = met_hourly['time'].dt.normalize()
met_hourly['hour'] = met_hourly['time'].dt.hour

# Compute daily aggregates from hourly (for RH, dew point, wind direction)
hourly_daily_agg = met_hourly.groupby('date_only').agg(
    rhum_mean=('rhum', 'mean'),
    rhum_max=('rhum', 'max'),
    rhum_min=('rhum', 'min'),
    dwpt_mean=('dwpt', 'mean'),
    wdir_mode=('wdir', lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else np.nan),
    wdir_mean=('wdir', 'mean'),
    wspd_mean=('wspd', 'mean'),
    wspd_max=('wspd', 'max'),
).reset_index()


print(f'Hourly met file: {MET_HOURLY_PATH}')
print(f'\nHourly met: {len(met_hourly)} records, {met_hourly["time"].min()} to {met_hourly["time"].max()}')
print(f'  Daily aggregates computed: {len(hourly_daily_agg)} days')


In [ ]:
# =============================================================================
# Load Source Apportionment + BC/EC Data
# =============================================================================

# Load factor contributions with Filter IDs
factors_df = load_etad_factors_with_filter_ids()
factors_df = factors_df.rename(columns={**FACTOR_TO_FRAC, **FACTOR_TO_CONC})

# Load aethalometer + filter measurements
aethalometer_data = load_aethalometer_data()
filter_data = load_filter_data()
filter_data = add_base_filter_id(filter_data)

df_aeth = aethalometer_data.get('Addis_Ababa')
bc_df = match_all_parameters('Addis_Ababa', 'ETAD', df_aeth, filter_data)

# Merge BC/EC measurements with factor contributions via base_filter_id
etad_filters = filter_data[filter_data['Site'] == 'ETAD'][['SampleDate', 'FilterId']].drop_duplicates()
etad_filters = etad_filters.rename(columns={'SampleDate': 'date', 'FilterId': 'base_filter_id'})
bc_df['date'] = pd.to_datetime(bc_df['date'])
etad_filters['date'] = pd.to_datetime(etad_filters['date'])
bc_with_id = pd.merge(bc_df, etad_filters, on='date', how='left')

factor_merge_cols = ['base_filter_id'] + frac_cols + conc_cols
available_merge_cols = [c for c in factor_merge_cols if c in factors_df.columns]
df = pd.merge(bc_with_id, factors_df[available_merge_cols], on='base_filter_id', how='left')

# Add time columns
df['Month'] = df['date'].dt.month
df['season'] = df['Month'].apply(get_season_3)
df['date_only'] = df['date'].dt.normalize()

# KF-based dominance
available_conc = [c for c in conc_cols if c in df.columns]
if available_conc:
    df['dominant_conc_col'] = df[available_conc].idxmax(axis=1)
    df['dominant_source'] = df['dominant_conc_col'].map(CONC_TO_SOURCE)
    df['kf_total'] = df[available_conc].sum(axis=1)
    df['dominant_kf_value'] = df[available_conc].max(axis=1)
    df['dominant_fraction'] = df['dominant_kf_value'] / df['kf_total']

# Grouped sources
if 'charcoal_conc' in df.columns and 'wood_conc' in df.columns:
    df['biomass_conc'] = df['charcoal_conc'].fillna(0) + df['wood_conc'].fillna(0)
if 'fossil_fuel_conc' in df.columns and 'polluted_marine_conc' in df.columns:
    df['fossil_comb_conc'] = df['fossil_fuel_conc'].fillna(0) + df['polluted_marine_conc'].fillna(0)

GROUPED_CONC_MAP = {
    'biomass_conc': 'biomass_burning',
    'fossil_comb_conc': 'fossil_fuel_combustion',
    'sea_salt_conc': 'sea_salt',
}
grouped_conc_cols = [c for c in ['biomass_conc', 'fossil_comb_conc', 'sea_salt_conc'] if c in df.columns]
df['dominant_source_grouped'] = df[grouped_conc_cols].idxmax(axis=1).map(GROUPED_CONC_MAP)

print(f"\nSource-apportioned dataset: {len(df)} samples")
print(f"Dominant source distribution:\n{df['dominant_source'].value_counts()}")

In [ ]:
# =============================================================================
# Merge Met Data with Source Apportionment Data
# =============================================================================

# Merge daily met
met_merge_cols = ['date_only', 'tavg', 'tmin', 'tmax', 'prcp', 'wspd', 'pres', 'temp_range']
met_merge = met_daily[[c for c in met_merge_cols if c in met_daily.columns]].copy()
df = pd.merge(df, met_merge, on='date_only', how='left', suffixes=('', '_met'))

# Merge hourly-aggregated daily data (RH, dew point, wind direction)
df = pd.merge(df, hourly_daily_agg, on='date_only', how='left')

# Check coverage
met_matched = df['tavg'].notna().sum()
rh_matched = df['rhum_mean'].notna().sum()
wdir_matched = df['wdir_mean'].notna().sum()

print(f"Met data matched to source samples:")
print(f"  Temperature: {met_matched}/{len(df)} ({met_matched/len(df)*100:.0f}%)")
print(f"  Humidity:    {rh_matched}/{len(df)} ({rh_matched/len(df)*100:.0f}%)")
print(f"  Wind dir:    {wdir_matched}/{len(df)} ({wdir_matched/len(df)*100:.0f}%)")
print(f"  Precip:      {df['prcp'].notna().sum()}/{len(df)} ({df['prcp'].notna().sum()/len(df)*100:.0f}%)")

# Quick sanity check
print(f"\nSample of merged data:")
check_cols = ['date', 'dominant_source', 'kf_total', 'tavg', 'prcp', 'rhum_mean', 'wdir_mean']
check_cols = [c for c in check_cols if c in df.columns]
print(df[check_cols].dropna().head(10).to_string(index=False))

---

# Section 1: Precipitation Washout by Source Type

**Question**: Does rain suppress biomass burning BC more than fossil fuel BC?  
**Relevance**: Ann asked for a write-up on "higher concentrations during rain" and seasonal patterns.

---

In [ ]:
# =============================================================================
# 1A: Precipitation Categories × Source Concentrations
# =============================================================================

# Define precipitation bins
prcp_bins = [0, 0.1, 2, 10, 50]
prcp_labels = ['Dry\n(0 mm)', 'Light\n(0.1–2 mm)', 'Moderate\n(2–10 mm)', 'Heavy\n(>10 mm)']

df_prcp = df.dropna(subset=['prcp', 'kf_total']).copy()
df_prcp['prcp_cat'] = pd.cut(df_prcp['prcp'], bins=[-0.01] + prcp_bins[1:],
                              labels=prcp_labels, include_lowest=True)

print("Samples per precipitation category:")
print(df_prcp['prcp_cat'].value_counts().sort_index())

# --- Figure: Total KF concentration by precip category ---
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Panel 1: Total KF by precip
ax = axes[0]
box_data = [df_prcp[df_prcp['prcp_cat'] == cat]['kf_total'].dropna() for cat in prcp_labels]
bp = ax.boxplot(box_data, labels=prcp_labels, patch_artist=True, widths=0.6,
                medianprops=dict(color='red', linewidth=2))
precip_colors = ['#FFF3E0', '#FFE0B2', '#90CAF9', '#1565C0']
for patch, color in zip(bp['boxes'], precip_colors):
    patch.set_facecolor(color)
ax.set_ylabel('Total KF Concentration (µg/m³)', fontsize=FONT['axis'])
ax.set_title('Total Source Concentration\nvs Precipitation', fontsize=FONT['title'])
ax.grid(True, axis='y', alpha=0.3)

# Add sample counts
for i, cat in enumerate(prcp_labels):
    n = len(box_data[i])
    ax.text(i+1, ax.get_ylim()[1]*0.95, f'n={n}', ha='center',
            fontsize=FONT['annot'], fontweight='bold')

# Panel 2: Individual source concentrations by precip (grouped bar)
ax = axes[1]
source_conc_cols = ['charcoal_conc', 'wood_conc', 'fossil_fuel_conc', 'polluted_marine_conc']
source_labels_short = ['Charcoal', 'Wood', 'Fossil Fuel', 'Poll. Marine']
source_colors_list = [SOURCE_CATEGORIES[s]['color'] for s in ['charcoal', 'wood', 'fossil_fuel', 'polluted_marine']]

x = np.arange(len(prcp_labels))
width = 0.18
for i, (col, lbl, clr) in enumerate(zip(source_conc_cols, source_labels_short, source_colors_list)):
    if col in df_prcp.columns:
        means = [df_prcp[df_prcp['prcp_cat'] == cat][col].mean() for cat in prcp_labels]
        sems = [df_prcp[df_prcp['prcp_cat'] == cat][col].sem() for cat in prcp_labels]
        ax.bar(x + i*width - width*1.5, means, width, yerr=sems,
               label=lbl, color=clr, edgecolor='black', capsize=3)

ax.set_xticks(x)
ax.set_xticklabels(prcp_labels, fontsize=FONT['tick'])
ax.set_ylabel('Mean Concentration (µg/m³)', fontsize=FONT['axis'])
ax.set_title('Source Concentrations\nby Precipitation Level', fontsize=FONT['title'])
ax.legend(fontsize=FONT['legend'] - 1)
ax.grid(True, axis='y', alpha=0.3)

# Panel 3: Grouped sources by precip
ax = axes[2]
grouped_cols_plot = ['biomass_conc', 'fossil_comb_conc']
grouped_labels_plot = ['Biomass Burning', 'Fossil Fuel Comb.']
grouped_colors_plot = [GROUPED_SOURCE_CATEGORIES['biomass_burning']['color'],
                       GROUPED_SOURCE_CATEGORIES['fossil_fuel_combustion']['color']]

width = 0.3
for i, (col, lbl, clr) in enumerate(zip(grouped_cols_plot, grouped_labels_plot, grouped_colors_plot)):
    if col in df_prcp.columns:
        means = [df_prcp[df_prcp['prcp_cat'] == cat][col].mean() for cat in prcp_labels]
        sems = [df_prcp[df_prcp['prcp_cat'] == cat][col].sem() for cat in prcp_labels]
        ax.bar(x + i*width - width*0.5, means, width, yerr=sems,
               label=lbl, color=clr, edgecolor='black', capsize=3)

ax.set_xticks(x)
ax.set_xticklabels(prcp_labels, fontsize=FONT['tick'])
ax.set_ylabel('Mean Concentration (µg/m³)', fontsize=FONT['axis'])
ax.set_title('Grouped Sources\nby Precipitation Level', fontsize=FONT['title'])
ax.legend(fontsize=FONT['legend'])
ax.grid(True, axis='y', alpha=0.3)

plt.suptitle('Precipitation Washout Effect on Aerosol Sources',
             fontsize=FONT['title'] + 2, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_precip_washout_by_source.png'),
            dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 1B: Washout Efficiency — Percent Reduction from Dry to Rainy Days
# =============================================================================

print("=" * 80)
print("WASHOUT ANALYSIS: % Concentration Change from Dry to Rainy Days")
print("=" * 80)

dry_mask = df_prcp['prcp_cat'] == prcp_labels[0]
rain_masks = {
    'Light Rain': df_prcp['prcp_cat'] == prcp_labels[1],
    'Moderate Rain': df_prcp['prcp_cat'] == prcp_labels[2],
    'Heavy Rain': df_prcp['prcp_cat'] == prcp_labels[3],
}

all_conc_cols = source_conc_cols + grouped_cols_plot + ['kf_total']
all_conc_labels = source_labels_short + grouped_labels_plot + ['Total KF']

print(f"\n{'Source':<25s} {'Dry Mean':>10s}", end='')
for rain_name in rain_masks:
    print(f" {rain_name+' Mean':>14s} {'Δ%':>7s}", end='')
print()
print("-" * 100)

for col, lbl in zip(all_conc_cols, all_conc_labels):
    if col not in df_prcp.columns:
        continue
    dry_mean = df_prcp.loc[dry_mask, col].mean()
    print(f"{lbl:<25s} {dry_mean:10.3f}", end='')
    for rain_name, rain_mask in rain_masks.items():
        rain_mean = df_prcp.loc[rain_mask, col].mean()
        pct_change = ((rain_mean - dry_mean) / dry_mean * 100) if dry_mean > 0 else np.nan
        print(f" {rain_mean:14.3f} {pct_change:+6.1f}%", end='')
    print()

print("\n(Negative % = washout / suppression; Positive % = enhancement)")

In [ ]:
# =============================================================================
# 1C: Precipitation × Season × Source Interaction
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5.5))

for idx, season in enumerate(SEASONS_ORDER):
    ax = axes[idx]
    season_data = df_prcp[df_prcp['season'] == season]
    
    if len(season_data) < 5:
        ax.text(0.5, 0.5, f'{season}\nInsufficient data', transform=ax.transAxes,
                ha='center', va='center')
        continue
    
    # Compute x-axis limit for this season panel
    x_max = season_data['prcp'].max() * 1.1 if len(season_data) > 0 else 1
    
    # Scatter + per-source regression lines
    for source in [s for s in SOURCE_ORDER if s != 'sea_salt']:
        info = SOURCE_CATEGORIES[source]
        mask = season_data['dominant_source'] == source
        subset = season_data[mask].dropna(subset=['prcp', 'kf_total'])
        if len(subset) > 0:
            ax.scatter(subset['prcp'], subset['kf_total'],
                      c=info['color'], marker=info['marker'], s=50, alpha=0.7,
                      edgecolors='black', linewidths=0.5, label=info['label'])
            
            # Per-source regression line (only if x values are not all identical)
            if len(subset) >= 3 and subset['prcp'].nunique() > 1:
                sl, intc, r_val, p_val, _ = stats.linregress(subset['prcp'], subset['kf_total'])
                x_line = np.linspace(0, x_max, 100)
                ax.plot(x_line, sl * x_line + intc, color=info['color'],
                       linewidth=2, alpha=0.8)
    
    # Overall trend line (dashed black)
    valid = season_data.dropna(subset=['prcp', 'kf_total'])
    if len(valid) >= 5 and valid['prcp'].nunique() > 1:
        slope, intercept, r, p, _ = stats.linregress(valid['prcp'], valid['kf_total'])
        x_line = np.linspace(0, x_max, 100)
        ax.plot(x_line, slope * x_line + intercept, 'k--', linewidth=1.5, alpha=0.7,
               label='Overall')
        sign = '+' if intercept >= 0 else '-'
        ax.text(0.95, 0.95, f'Overall: y={slope:.2f}x {sign} {abs(intercept):.2f}\nr={r:.2f}, p={p:.3f}',
               transform=ax.transAxes, ha='right', va='top',
               fontsize=FONT['annot'], bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    ax.set_xlabel('Daily Precipitation (mm)', fontsize=FONT['axis'])
    ax.set_ylabel('Total KF (µg/m³)' if idx == 0 else '', fontsize=FONT['axis'])
    ax.set_title(f'{season}\n(n={len(season_data)})', fontsize=FONT['title'],
                color=SEASON_COLORS[season], fontweight='bold')
    ax.grid(True, alpha=0.3)
    if idx == 2:
        ax.legend(fontsize=FONT['legend'] - 2, loc='upper right')

plt.suptitle('Precipitation vs Total Source Concentration by Season',
             fontsize=FONT['title'] + 2, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_precip_vs_kf_by_season.png'),
            dpi=200, bbox_inches='tight')
plt.show()

---

# Section 2: Wind Direction × Source Type

**Question**: Do certain source types correlate with specific wind directions?  
**Expectation**: Polluted marine might arrive from easterly/SE directions; biomass may be local.

---

In [ ]:
# =============================================================================
# Wind Rose Helper Function
# =============================================================================

def wind_rose(ax, directions, speeds=None, weights=None, n_bins=16, title='',
              color='steelblue', speed_bins=None):
    """
    Draw a wind rose on a polar axis.
    
    Parameters:
    -----------
    ax : polar axes
    directions : array of wind directions in degrees
    speeds : optional array of wind speeds (for speed-binned coloring)
    weights : optional weights for each observation
    n_bins : number of direction bins (default 16 = 22.5° each)
    """
    dir_bins = np.linspace(0, 360, n_bins + 1)
    bin_width = 2 * np.pi / n_bins
    
    valid = np.isfinite(directions)
    dirs = np.array(directions)[valid]
    
    if speeds is not None and speed_bins is not None:
        spds = np.array(speeds)[valid]
        # Stack speed bins
        colors_speed = plt.cm.YlOrRd(np.linspace(0.2, 0.9, len(speed_bins) - 1))
        bottom = np.zeros(n_bins)
        
        for j in range(len(speed_bins) - 1):
            spd_mask = (spds >= speed_bins[j]) & (spds < speed_bins[j+1])
            counts, _ = np.histogram(dirs[spd_mask], bins=dir_bins)
            freq = counts / len(dirs) * 100
            theta = np.deg2rad(dir_bins[:-1] + 360/(2*n_bins))  # center of each bin
            ax.bar(theta, freq, width=bin_width, bottom=bottom,
                   color=colors_speed[j], edgecolor='white', linewidth=0.5,
                   label=f'{speed_bins[j]}–{speed_bins[j+1]} km/h')
            bottom += freq
    else:
        counts, _ = np.histogram(dirs, bins=dir_bins)
        freq = counts / len(dirs) * 100
        theta = np.deg2rad(dir_bins[:-1] + 360/(2*n_bins))
        ax.bar(theta, freq, width=bin_width, color=color,
               edgecolor='white', linewidth=0.5, alpha=0.8)
    
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_thetagrids([0, 45, 90, 135, 180, 225, 270, 315],
                       ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'])
    ax.set_title(title, fontsize=FONT['title'], fontweight='bold', pad=20)

print("Wind rose function defined")

In [ ]:
# =============================================================================
# 2A: Overall Wind Rose with Speed Bins (hourly data)
# =============================================================================

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='polar')

valid_hourly = met_hourly.dropna(subset=['wdir', 'wspd'])
wind_rose(ax, valid_hourly['wdir'].values, speeds=valid_hourly['wspd'].values,
          speed_bins=[0, 5, 10, 15, 25, 180],
          title=f'Addis Ababa Wind Rose\n(n={len(valid_hourly):,} hourly obs)')
ax.legend(loc='lower right', bbox_to_anchor=(1.3, -0.05), fontsize=FONT['legend'])

plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_wind_rose_overall.png'),
            dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 2B: Wind Rose by Dominant Source Type
# =============================================================================

# For each filter sample day, get the hourly wind data from that day
# and plot the wind rose colored by which source dominated that filter

df_wind = df.dropna(subset=['wdir_mean', 'dominant_source']).copy()
sources_to_plot = [s for s in SOURCE_ORDER if s != 'sea_salt']

fig, axes = plt.subplots(1, 4, figsize=(20, 5),
                          subplot_kw=dict(projection='polar'))

for idx, source in enumerate(sources_to_plot):
    ax = axes[idx]
    info = SOURCE_CATEGORIES[source]
    
    # Get dates when this source dominated
    source_dates = df_wind[df_wind['dominant_source'] == source]['date_only'].values
    
    # Get hourly wind data for those dates
    hourly_subset = met_hourly[met_hourly['date_only'].isin(source_dates)]
    hourly_valid = hourly_subset.dropna(subset=['wdir'])
    
    if len(hourly_valid) > 10:
        wind_rose(ax, hourly_valid['wdir'].values,
                 title=f"{info['label']}\n(n={len(source_dates)} days)",
                 color=info['color'])
    else:
        ax.set_title(f"{info['label']}\n(insufficient data)",
                    fontsize=FONT['title'])

plt.suptitle('Wind Direction on Days Dominated by Each Source',
             fontsize=FONT['title'] + 2, fontweight='bold', y=1.08)
plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_wind_rose_by_source.png'),
            dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 2C: Wind Rose by Season
# =============================================================================

met_hourly['Month'] = met_hourly['time'].dt.month
met_hourly['season'] = met_hourly['Month'].apply(get_season_3)

fig, axes = plt.subplots(1, 3, figsize=(18, 6),
                          subplot_kw=dict(projection='polar'))

for idx, season in enumerate(SEASONS_ORDER):
    ax = axes[idx]
    season_data = met_hourly[met_hourly['season'] == season].dropna(subset=['wdir', 'wspd'])
    
    wind_rose(ax, season_data['wdir'].values, speeds=season_data['wspd'].values,
             speed_bins=[0, 5, 10, 15, 25, 180],
             title=f"{season}\n(n={len(season_data):,})")

axes[-1].legend(loc='lower right', bbox_to_anchor=(1.4, -0.05), fontsize=FONT['legend'])
plt.suptitle('Seasonal Wind Patterns — Addis Ababa',
             fontsize=FONT['title'] + 2, fontweight='bold', y=1.08)
plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_wind_rose_by_season.png'),
            dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 2D: Mean Wind Direction & Concentration Polar Scatter
# =============================================================================

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='polar')

df_polar = df.dropna(subset=['wdir_mean', 'kf_total', 'dominant_source']).copy()

for source in [s for s in SOURCE_ORDER if s != 'sea_salt']:
    info = SOURCE_CATEGORIES[source]
    mask = df_polar['dominant_source'] == source
    subset = df_polar[mask]
    if len(subset) == 0:
        continue
    
    theta = np.deg2rad(subset['wdir_mean'].values)
    r = subset['kf_total'].values
    
    ax.scatter(theta, r, c=info['color'], marker=info['marker'],
              s=60, alpha=0.7, edgecolors='black', linewidths=0.5,
              label=f"{info['label']} (n={len(subset)})")

ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)
ax.set_thetagrids([0, 45, 90, 135, 180, 225, 270, 315],
                   ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'])
ax.set_title('Source Concentration vs Mean Wind Direction\n(radius = total KF µg/m³)',
             fontsize=FONT['title'], fontweight='bold', pad=25)
ax.legend(loc='lower right', bbox_to_anchor=(1.35, -0.05), fontsize=FONT['legend'])

plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_polar_scatter_source_wind.png'),
            dpi=200, bbox_inches='tight')
plt.show()

---

# Section 3: Temperature Range vs Concentration (Inversion Proxy)

**Question**: Does a narrow diurnal temperature range (Tmax − Tmin) → atmospheric stability → higher pollutant concentrations?  
**Rationale**: Smaller ΔT suggests weaker mixing / stronger inversions, trapping aerosols near the surface.

---

In [ ]:
# =============================================================================
# 3A: Diurnal Temperature Range vs Total KF Concentration
# =============================================================================

df_temp = df.dropna(subset=['temp_range', 'kf_total']).copy()

print(f"Samples with both temp range and KF data: {len(df_temp)}")
print(f"Temperature range: {df_temp['temp_range'].min():.1f}°C to {df_temp['temp_range'].max():.1f}°C")
print(f"Mean ΔT: {df_temp['temp_range'].mean():.1f}°C")

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# X-axis range for regression lines
dt_min = df_temp['temp_range'].min()
dt_max = df_temp['temp_range'].max()

# Panel 1: Scatter — ΔT vs Total KF, colored by season with per-season regression
ax = axes[0]
for season in SEASONS_ORDER:
    mask = df_temp['season'] == season
    subset = df_temp[mask]
    ax.scatter(subset['temp_range'], subset['kf_total'],
              c=SEASON_COLORS[season], s=50, alpha=0.7,
              edgecolors='black', linewidths=0.5,
              label=f"{season} (n={len(subset)})")
    
    # Per-season regression line
    if len(subset) >= 3:
        sl, intc, r_val, _, _ = stats.linregress(subset['temp_range'], subset['kf_total'])
        x_line = np.linspace(dt_min, dt_max, 100)
        ax.plot(x_line, sl * x_line + intc, color=SEASON_COLORS[season],
               linewidth=2, alpha=0.8)

# Overall trend (dashed black)
slope, intercept, r, p, _ = stats.linregress(df_temp['temp_range'], df_temp['kf_total'])
x_line = np.linspace(dt_min, dt_max, 100)
ax.plot(x_line, slope * x_line + intercept, 'k--', linewidth=2, alpha=0.7,
       label='Overall')
sign = '+' if intercept >= 0 else '-'
ax.text(0.05, 0.95, f'Overall: y={slope:.3f}x {sign} {abs(intercept):.3f}\nr = {r:.3f}, p = {p:.3f}',
        transform=ax.transAxes, va='top', fontsize=FONT['annot'],
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

ax.set_xlabel('Diurnal Temperature Range (°C)', fontsize=FONT['axis'])
ax.set_ylabel('Total KF Concentration (µg/m³)', fontsize=FONT['axis'])
ax.set_title('ΔT vs Total Concentration', fontsize=FONT['title'])
ax.legend(fontsize=FONT['legend'] - 1)
ax.grid(True, alpha=0.3)

# Panel 2: Scatter — ΔT vs Total KF, colored by dominant source with per-source regression
ax = axes[1]
legend_elements = []
for source in [s for s in SOURCE_ORDER if s != 'sea_salt']:
    info = SOURCE_CATEGORIES[source]
    mask = df_temp['dominant_source'] == source
    subset = df_temp[mask]
    if len(subset) > 0:
        ax.scatter(subset['temp_range'], subset['kf_total'],
                  c=info['color'], marker=info['marker'], s=50, alpha=0.7,
                  edgecolors='black', linewidths=0.5)
        
        # Per-source regression line
        if len(subset) >= 3:
            sl, intc, r_val, _, _ = stats.linregress(subset['temp_range'], subset['kf_total'])
            r2 = r_val**2
            x_line = np.linspace(dt_min, dt_max, 100)
            ax.plot(x_line, sl * x_line + intc, color=info['color'],
                   linewidth=2.5, alpha=0.8)
            sign = '+' if intc >= 0 else '-'
            legend_elements.append(
                Line2D([0], [0], marker=info['marker'], color=info['color'],
                       markeredgecolor='black', markersize=8, linewidth=2,
                       label=f"{info['label']}: y={sl:.2f}x {sign} {abs(intc):.2f}, R²={r2:.2f} (n={len(subset)})"))
        else:
            legend_elements.append(
                Line2D([0], [0], marker=info['marker'], color=info['color'],
                       markeredgecolor='black', markersize=8, linewidth=0,
                       label=f"{info['label']} (n={len(subset)})"))

ax.set_xlabel('Diurnal Temperature Range (°C)', fontsize=FONT['axis'])
ax.set_ylabel('Total KF Concentration (µg/m³)', fontsize=FONT['axis'])
ax.set_title('ΔT vs Concentration by Source', fontsize=FONT['title'])
ax.legend(handles=legend_elements, fontsize=FONT['legend'] - 2, loc='upper left')
ax.grid(True, alpha=0.3)

# Panel 3: Box plot — ΔT binned
ax = axes[2]
dt_bins = pd.qcut(df_temp['temp_range'], q=4, duplicates='drop')
df_temp['dt_quartile'] = dt_bins

quartile_labels = [str(q) for q in sorted(df_temp['dt_quartile'].dropna().unique())]
box_data = [df_temp[df_temp['dt_quartile'] == q]['kf_total'].dropna()
            for q in sorted(df_temp['dt_quartile'].dropna().unique())]

bp = ax.boxplot(box_data, labels=[f'Q{i+1}' for i in range(len(box_data))],
                patch_artist=True, widths=0.6,
                medianprops=dict(color='red', linewidth=2))

dt_colors = plt.cm.RdYlBu_r(np.linspace(0.2, 0.8, len(box_data)))
for patch, color in zip(bp['boxes'], dt_colors):
    patch.set_facecolor(color)

# Add quartile ranges as labels
for i, q in enumerate(sorted(df_temp['dt_quartile'].dropna().unique())):
    n = len(box_data[i])
    ax.text(i+1, ax.get_ylim()[0] - 0.05*(ax.get_ylim()[1]-ax.get_ylim()[0]),
            f'{q}\nn={n}', ha='center', fontsize=FONT['annot'] - 2, va='top')

ax.set_ylabel('Total KF Concentration (µg/m³)', fontsize=FONT['axis'])
ax.set_title('Concentration by ΔT Quartile\n(Q1=narrowest range)', fontsize=FONT['title'])
ax.grid(True, axis='y', alpha=0.3)

plt.suptitle('Atmospheric Stability Proxy: Diurnal Temperature Range × Source Concentration',
             fontsize=FONT['title'] + 1, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_temp_range_vs_concentration.png'),
            dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 3B: Per-Source Correlation with ΔT
# =============================================================================

print("=" * 80)
print("TEMPERATURE RANGE CORRELATION BY SOURCE")
print("=" * 80)

print(f"\n{'Source':<25s} {'r':>8s} {'p-value':>10s} {'slope':>8s} {'n':>5s} {'Interpretation'}")
print("-" * 85)

for col, source_key in CONC_TO_SOURCE.items():
    if col not in df_temp.columns:
        continue
    valid = df_temp.dropna(subset=['temp_range', col])
    if len(valid) < 5:
        continue
    
    r, p = stats.pearsonr(valid['temp_range'], valid[col])
    slope, _, _, _, _ = stats.linregress(valid['temp_range'], valid[col])
    label = SOURCE_CATEGORIES[source_key]['label']
    
    interp = ''
    if p < 0.05:
        if r > 0:
            interp = '↑ Higher on well-mixed days'
        else:
            interp = '↑ Higher on stable/stagnant days'
    else:
        interp = 'Not significant'
    
    print(f"{label:<25s} {r:8.3f} {p:10.4f} {slope:8.4f} {len(valid):5d} {interp}")

# Also grouped
print(f"\n--- Grouped Sources ---")
for col, label in [('biomass_conc', 'Biomass Burning'), ('fossil_comb_conc', 'Fossil Fuel Comb.')]:
    if col not in df_temp.columns:
        continue
    valid = df_temp.dropna(subset=['temp_range', col])
    if len(valid) < 5:
        continue
    r, p = stats.pearsonr(valid['temp_range'], valid[col])
    slope, _, _, _, _ = stats.linregress(valid['temp_range'], valid[col])
    interp = '↑ Stable' if (p < 0.05 and r < 0) else ('↑ Mixed' if (p < 0.05 and r > 0) else 'NS')
    print(f"{label:<25s} {r:8.3f} {p:10.4f} {slope:8.4f} {len(valid):5d} {interp}")

In [ ]:
# =============================================================================
# 3C: Multi-Met Variable Correlation Heatmap
# =============================================================================

met_vars = ['tavg', 'tmin', 'tmax', 'temp_range', 'prcp', 'wspd_mean', 'rhum_mean', 'pres']
source_vars = [c for c in conc_cols + ['kf_total', 'biomass_conc', 'fossil_comb_conc']
               if c in df.columns]

met_available = [c for c in met_vars if c in df.columns]
corr_cols = met_available + source_vars

corr_df = df[corr_cols].dropna()
print(f"Correlation matrix computed from {len(corr_df)} complete samples")

corr_matrix = corr_df.corr()

# Extract just the met × source block
corr_block = corr_matrix.loc[met_available, source_vars]

# Rename for readability
met_rename = {
    'tavg': 'T avg', 'tmin': 'T min', 'tmax': 'T max',
    'temp_range': 'ΔT (Tmax−Tmin)', 'prcp': 'Precipitation',
    'wspd_mean': 'Wind Speed', 'rhum_mean': 'Rel. Humidity', 'pres': 'Pressure'
}
source_rename = {
    'charcoal_conc': 'Charcoal', 'wood_conc': 'Wood',
    'fossil_fuel_conc': 'Fossil Fuel', 'polluted_marine_conc': 'Poll. Marine',
    'sea_salt_conc': 'Sea Salt', 'kf_total': 'Total KF',
    'biomass_conc': 'Biomass (grp)', 'fossil_comb_conc': 'Fossil (grp)',
}

corr_block = corr_block.rename(index=met_rename, columns=source_rename)

fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(corr_block, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            vmin=-0.6, vmax=0.6, linewidths=0.5, ax=ax,
            annot_kws={'fontsize': FONT['annot']})
ax.set_title('Meteorology × Source Concentration Correlation Matrix',
             fontsize=FONT['title'], fontweight='bold', pad=15)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=FONT['tick'])
ax.set_yticklabels(ax.get_yticklabels(), fontsize=FONT['tick'])

plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_source_correlation_heatmap.png'),
            dpi=200, bbox_inches='tight')
plt.show()

---

# Section 4: Humidity Effect on Measurement Agreement

**Question**: Does relative humidity explain scatter in HIPS vs FTIR EC?  
**Rationale**: HIPS measures optical absorption (affected by particle water uptake),
while FTIR is a mass-based method. Humid conditions may cause divergence.

---

In [ ]:
# =============================================================================
# 4A: HIPS vs FTIR EC — Colored by Daily Mean RH
# =============================================================================

PAIRS_FOR_RH = [
    ('ftir_ec', 'hips_fabs', 'FTIR EC (µg/m³)', 'HIPS Fabs/MAC (µg/m³)'),
    ('ftir_ec', 'ir_bcc',    'FTIR EC (µg/m³)', 'Aeth IR BCc (µg/m³)'),
    ('hips_fabs', 'ir_bcc',  'HIPS Fabs/MAC (µg/m³)', 'Aeth IR BCc (µg/m³)'),
]

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

for idx, (x_col, y_col, x_label, y_label) in enumerate(PAIRS_FOR_RH):
    ax = axes[idx]
    
    plot_df = df.dropna(subset=[x_col, y_col, 'rhum_mean']).copy()
    
    if len(plot_df) < 5:
        ax.text(0.5, 0.5, 'Insufficient data', transform=ax.transAxes, ha='center')
        continue
    
    sc = ax.scatter(plot_df[x_col], plot_df[y_col], c=plot_df['rhum_mean'],
                   cmap='YlGnBu', s=60, alpha=0.8, edgecolors='black', linewidths=0.5,
                   vmin=30, vmax=95)
    
    # 1:1 line
    all_vals = np.concatenate([plot_df[x_col].values, plot_df[y_col].values])
    lim = np.nanpercentile(all_vals, 99) * 1.1
    ax.plot([0, lim], [0, lim], 'k--', alpha=0.5, linewidth=1)
    
    # Regression
    slope, intercept, r, p, _ = stats.linregress(plot_df[x_col], plot_df[y_col])
    x_line = np.linspace(0, lim, 100)
    ax.plot(x_line, slope * x_line + intercept, 'r-', linewidth=2, alpha=0.7)
    ax.text(0.05, 0.95, f'y = {slope:.2f}x + {intercept:.2f}\nR² = {r**2:.3f}\nn = {len(plot_df)}',
            transform=ax.transAxes, va='top', fontsize=FONT['annot'],
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    ax.set_xlabel(x_label, fontsize=FONT['axis'])
    ax.set_ylabel(y_label, fontsize=FONT['axis'])
    ax.set_xlim(0, lim)
    ax.set_ylim(0, lim)
    ax.set_aspect('equal', adjustable='box')
    ax.grid(True, alpha=0.3)
    
    cbar = plt.colorbar(sc, ax=ax, shrink=0.8, pad=0.02)
    cbar.set_label('Daily Mean RH (%)', fontsize=FONT['annot'])

plt.suptitle('BC/EC Method Comparison Colored by Relative Humidity',
             fontsize=FONT['title'] + 1, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_rh_effect_on_agreement.png'),
            dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 4B: Split Regressions — Dry vs Humid Days
# =============================================================================

# Split at median RH
df_rh = df.dropna(subset=['rhum_mean']).copy()
rh_median = df_rh['rhum_mean'].median()
df_rh['rh_category'] = np.where(df_rh['rhum_mean'] <= rh_median, 'Dry', 'Humid')

print(f"RH median: {rh_median:.1f}%")
print(f"Dry days (RH ≤ {rh_median:.0f}%): {(df_rh['rh_category'] == 'Dry').sum()}")
print(f"Humid days (RH > {rh_median:.0f}%): {(df_rh['rh_category'] == 'Humid').sum()}")

fig, axes = plt.subplots(2, 3, figsize=(20, 12))

rh_colors = {'Dry': '#E67E22', 'Humid': '#3498DB'}

for col_idx, (x_col, y_col, x_label, y_label) in enumerate(PAIRS_FOR_RH):
    for row_idx, rh_cat in enumerate(['Dry', 'Humid']):
        ax = axes[row_idx, col_idx]
        subset = df_rh[(df_rh['rh_category'] == rh_cat)].dropna(subset=[x_col, y_col])
        
        if len(subset) < 5:
            ax.text(0.5, 0.5, f'{rh_cat}\nInsufficient data',
                    transform=ax.transAxes, ha='center')
            continue
        
        ax.scatter(subset[x_col], subset[y_col], c=rh_colors[rh_cat],
                  s=50, alpha=0.7, edgecolors='black', linewidths=0.5)
        
        slope, intercept, r, p, _ = stats.linregress(subset[x_col], subset[y_col])
        all_vals = np.concatenate([subset[x_col].values, subset[y_col].values])
        lim = np.nanpercentile(all_vals, 99) * 1.1
        x_line = np.linspace(0, lim, 100)
        ax.plot(x_line, slope * x_line + intercept, color=rh_colors[rh_cat],
               linewidth=2, alpha=0.8)
        ax.plot([0, lim], [0, lim], 'k--', alpha=0.4, linewidth=1)
        
        rh_range = subset['rhum_mean']
        ax.text(0.05, 0.95,
                f'{rh_cat} (RH: {rh_range.min():.0f}–{rh_range.max():.0f}%)\n'
                f'y = {slope:.3f}x + {intercept:.3f}\n'
                f'R² = {r**2:.3f}, n = {len(subset)}',
                transform=ax.transAxes, va='top', fontsize=FONT['annot'],
                bbox=dict(boxstyle='round', facecolor=rh_colors[rh_cat], alpha=0.2))
        
        ax.set_xlabel(x_label, fontsize=FONT['axis'] - 1)
        ax.set_ylabel(y_label, fontsize=FONT['axis'] - 1)
        ax.set_xlim(0, lim)
        ax.set_ylim(0, lim)
        ax.set_aspect('equal', adjustable='box')
        ax.grid(True, alpha=0.3)
        
        if col_idx == 0:
            ax.set_ylabel(f'{rh_cat.upper()} Days\n{y_label}', fontsize=FONT['axis'])

plt.suptitle(f'BC/EC Measurement Agreement: Dry (RH ≤ {rh_median:.0f}%) vs Humid (RH > {rh_median:.0f}%)',
             fontsize=FONT['title'] + 1, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_dry_vs_humid_regressions.png'),
            dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 4C: Systematic RH Threshold Sweep — How Does Slope Change with Humidity?
# =============================================================================

rh_thresholds = np.arange(35, 90, 5)

fig, axes = plt.subplots(1, 3, figsize=(18, 5.5))

for idx, (x_col, y_col, x_label, y_label) in enumerate(PAIRS_FOR_RH):
    ax = axes[idx]
    
    slopes_below = []
    slopes_above = []
    r2_below = []
    r2_above = []
    
    for rh_thresh in rh_thresholds:
        below = df_rh[df_rh['rhum_mean'] <= rh_thresh].dropna(subset=[x_col, y_col])
        above = df_rh[df_rh['rhum_mean'] > rh_thresh].dropna(subset=[x_col, y_col])
        
        if len(below) >= 5:
            s, _, r, _, _ = stats.linregress(below[x_col], below[y_col])
            slopes_below.append(s)
            r2_below.append(r**2)
        else:
            slopes_below.append(np.nan)
            r2_below.append(np.nan)
        
        if len(above) >= 5:
            s, _, r, _, _ = stats.linregress(above[x_col], above[y_col])
            slopes_above.append(s)
            r2_above.append(r**2)
        else:
            slopes_above.append(np.nan)
            r2_above.append(np.nan)
    
    ax.plot(rh_thresholds, slopes_below, 'o-', color='#E67E22', linewidth=2,
            markersize=6, label='Slope (RH ≤ threshold)')
    ax.plot(rh_thresholds, slopes_above, 's-', color='#3498DB', linewidth=2,
            markersize=6, label='Slope (RH > threshold)')
    
    # Secondary axis for R²
    ax2 = ax.twinx()
    ax2.plot(rh_thresholds, r2_below, 'o--', color='#E67E22', linewidth=1,
             markersize=4, alpha=0.5)
    ax2.plot(rh_thresholds, r2_above, 's--', color='#3498DB', linewidth=1,
             markersize=4, alpha=0.5)
    ax2.set_ylabel('R² (dashed)', fontsize=FONT['axis'] - 2, alpha=0.6)
    ax2.set_ylim(0, 1)
    
    ax.set_xlabel('RH Threshold (%)', fontsize=FONT['axis'])
    ax.set_ylabel('Regression Slope', fontsize=FONT['axis'])
    ax.set_title(f'{y_label.split("(")[0].strip()}\nvs {x_label.split("(")[0].strip()}',
                fontsize=FONT['title'])
    ax.legend(fontsize=FONT['legend'] - 1, loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=1.0, color='gray', linestyle=':', alpha=0.5, label='slope=1')

plt.suptitle('How Does Measurement Agreement Slope Change with Humidity?',
             fontsize=FONT['title'] + 1, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_rh_threshold_sweep_slopes.png'),
            dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 4D: Residual Analysis — Does RH Explain Regression Residuals?
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5.5))

for idx, (x_col, y_col, x_label, y_label) in enumerate(PAIRS_FOR_RH):
    ax = axes[idx]
    
    plot_df = df_rh.dropna(subset=[x_col, y_col, 'rhum_mean']).copy()
    if len(plot_df) < 10:
        ax.text(0.5, 0.5, 'Insufficient data', transform=ax.transAxes, ha='center')
        continue
    
    # Fit regression
    slope, intercept, _, _, _ = stats.linregress(plot_df[x_col], plot_df[y_col])
    plot_df['predicted'] = slope * plot_df[x_col] + intercept
    plot_df['residual'] = plot_df[y_col] - plot_df['predicted']
    
    # Scatter residual vs RH
    sc = ax.scatter(plot_df['rhum_mean'], plot_df['residual'],
                   c=plot_df['rhum_mean'], cmap='YlGnBu', s=50, alpha=0.7,
                   edgecolors='black', linewidths=0.5, vmin=30, vmax=95)
    ax.axhline(y=0, color='red', linestyle='--', linewidth=1.5)
    
    # Trend line
    r_res, p_res = stats.pearsonr(plot_df['rhum_mean'], plot_df['residual'])
    s_res, i_res, _, _, _ = stats.linregress(plot_df['rhum_mean'], plot_df['residual'])
    x_line = np.linspace(plot_df['rhum_mean'].min(), plot_df['rhum_mean'].max(), 100)
    ax.plot(x_line, s_res * x_line + i_res, 'k-', linewidth=2, alpha=0.6)
    
    ax.text(0.05, 0.95, f'r = {r_res:.3f}, p = {p_res:.3f}',
            transform=ax.transAxes, va='top', fontsize=FONT['annot'],
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    pair_label = f'{y_label.split("(")[0].strip()} vs {x_label.split("(")[0].strip()}'
    ax.set_xlabel('Daily Mean RH (%)', fontsize=FONT['axis'])
    ax.set_ylabel('Regression Residual (µg/m³)', fontsize=FONT['axis'])
    ax.set_title(f'Residuals: {pair_label}', fontsize=FONT['title'] - 1)
    ax.grid(True, alpha=0.3)

plt.suptitle('Do Regression Residuals Correlate with Relative Humidity?',
             fontsize=FONT['title'] + 1, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_rh_residual_analysis.png'),
            dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 4E: Split Regressions — Dry vs Humid (Threshold from 4C Sweep)
# =============================================================================
# From the 4C threshold sweep, slopes diverge most consistently around RH ~55%.
# This replaces the median-based split with a physically-motivated threshold.

RH_THRESHOLD = 55  # Identified from 4C sweep as inflection point

df_rh['rh_category_4c'] = np.where(df_rh['rhum_mean'] <= RH_THRESHOLD, 'Dry', 'Humid')

n_dry = (df_rh['rh_category_4c'] == 'Dry').sum()
n_humid = (df_rh['rh_category_4c'] == 'Humid').sum()
print(f"RH threshold (from 4C sweep): {RH_THRESHOLD}%")
print(f"Dry days (RH ≤ {RH_THRESHOLD}%): {n_dry}")
print(f"Humid days (RH > {RH_THRESHOLD}%): {n_humid}")

fig, axes = plt.subplots(2, 3, figsize=(20, 12))

rh_colors = {'Dry': '#E67E22', 'Humid': '#3498DB'}

for col_idx, (x_col, y_col, x_label, y_label) in enumerate(PAIRS_FOR_RH):
    # Compute consistent axis limit across both rows for this pair
    pair_data = df_rh.dropna(subset=[x_col, y_col])
    all_vals = np.concatenate([pair_data[x_col].values, pair_data[y_col].values])
    lim = np.nanpercentile(all_vals, 99) * 1.1

    for row_idx, rh_cat in enumerate(['Dry', 'Humid']):
        ax = axes[row_idx, col_idx]
        subset = df_rh[(df_rh['rh_category_4c'] == rh_cat)].dropna(subset=[x_col, y_col])

        if len(subset) < 5:
            ax.text(0.5, 0.5, f'{rh_cat}\nInsufficient data',
                    transform=ax.transAxes, ha='center')
            continue

        ax.scatter(subset[x_col], subset[y_col], c=rh_colors[rh_cat],
                  s=50, alpha=0.7, edgecolors='black', linewidths=0.5)

        slope, intercept, r, p, _ = stats.linregress(subset[x_col], subset[y_col])
        x_line = np.linspace(0, lim, 100)
        ax.plot(x_line, slope * x_line + intercept, color=rh_colors[rh_cat],
               linewidth=2, alpha=0.8)
        ax.plot([0, lim], [0, lim], 'k--', alpha=0.4, linewidth=1)

        rh_range = subset['rhum_mean']
        sign = '+' if intercept >= 0 else '-'
        ax.text(0.05, 0.95,
                f'{rh_cat} (RH: {rh_range.min():.0f}–{rh_range.max():.0f}%)\n'
                f'y = {slope:.3f}x {sign} {abs(intercept):.3f}\n'
                f'R² = {r**2:.3f}, n = {len(subset)}',
                transform=ax.transAxes, va='top', fontsize=FONT['annot'],
                bbox=dict(boxstyle='round', facecolor=rh_colors[rh_cat], alpha=0.2))

        ax.set_xlabel(x_label, fontsize=FONT['axis'] - 1)
        ax.set_ylabel(y_label, fontsize=FONT['axis'] - 1)
        ax.set_xlim(0, lim)
        ax.set_ylim(0, lim)
        ax.set_aspect('equal', adjustable='box')
        ax.grid(True, alpha=0.3)

        if col_idx == 0:
            ax.set_ylabel(f'{rh_cat.upper()} Days\n{y_label}', fontsize=FONT['axis'])

plt.suptitle(f'BC/EC Measurement Agreement: Dry (RH ≤ {RH_THRESHOLD}%) vs Humid (RH > {RH_THRESHOLD}%)\n'
             f'[Threshold from 4C sweep — slope inflection point]',
             fontsize=FONT['title'] + 1, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig(os.path.join(dirs['plots'], 'met_dry_vs_humid_regressions_4c_threshold.png'),
            dpi=200, bbox_inches='tight')
plt.show()

---

# Summary & Key Findings

---

In [ ]:
# =============================================================================
# Summary Statistics Table
# =============================================================================

print("=" * 80)
print("METEOROLOGY × SOURCE APPORTIONMENT — SUMMARY")
print("=" * 80)

print("\n1. PRECIPITATION WASHOUT")
print("   Check the grouped bar chart and washout table above.")
print("   → Which sources show strongest washout during heavy rain?")
print("   → Is biomass burning more affected than fossil fuel?")

print("\n2. WIND DIRECTION")
print("   Compare wind roses by source type.")
print("   → Does polluted marine show a distinct directional signature?")
print("   → How do seasonal wind patterns align with source shifts?")

print("\n3. TEMPERATURE RANGE (STABILITY PROXY)")
print("   Check correlation table and scatter plots.")
print("   → Negative r = higher concentrations on stable/stagnant days")
print("   → Positive r = higher on well-mixed days (transported sources?)")

print("\n4. HUMIDITY & MEASUREMENT AGREEMENT")
print("   Check split regressions and residual analysis.")
print("   → Does slope shift between dry and humid conditions?")
print("   → Are residuals correlated with RH (systematic bias)?")

print("\n" + "=" * 80)
print("Files saved to:", dirs['plots'])
print("=" * 80)